In [114]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:\Users\Mahek\Downloads\archive (3)\sentiment_train.csv')


In [115]:
df.head()

,sentence,label
0,Ok brokeback mountain is such a horrible movie.,0
1,Brokeback Mountain was so awesome.,1
2,friday hung out with kelsie and we went and sa...,0
3,I am going to start reading the Harry Potter s...,1
4,"Is it just me, or does Harry Potter suck?...",0


In [116]:
from sklearn.model_selection import train_test_split

train_text, temp_text, train_labels, temp_labels = train_test_split(df['sentence'], df['label'],
																	random_state = 2021,
																	test_size = 0.3,
																	stratify = df['label'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
																random_state = 2021,
																test_size = 0.5,
																stratify = temp_labels)


In [117]:
from transformers import AutoModel, BertTokenizerFast

bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')



In [118]:
import matplotlib.pyplot as plt
train_lens = [len(i.split()) for i in train_text]
plt.hist(train_lens)


(array([1504.,  701.,  572.,  521.,  164.,  215.,  117.,   63.,  104.,
           6.]),
 array([ 3. ,  6.5, 10. , 13.5, 17. , 20.5, 24. , 27.5, 31. , 34.5, 38. ]),
 <BarContainer object of 10 artists>)

In [119]:
# tokenize and encode sequences
import torch
pad_len = 17
# Instead of using pad_to_max_length, use padding='max_length' and max_length argument
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    padding='max_length',  # Use padding='max_length'
    max_length=pad_len,     # Specify the maximum length
    truncation=True
)

tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    padding='max_length',  # Use padding='max_length'
    max_length=pad_len,     # Specify the maximum length
    truncation=True
)

tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    padding='max_length',  # Use padding='max_length'
    max_length=pad_len,     # Specify the maximum length
    truncation=True
)


train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())


In [120]:
#freeze the pretrained layers
import torch
import torch.nn as nn 
for param in bert.parameters():
    param.requires_grad = False
  
#defining new layers
class BERT_architecture(nn.Module):
    def __init__(self, bert):
        
      super(BERT_architecture, self).__init__()
  
      self.bert = bert 
        
      # dropout layer
      self.dropout = nn.Dropout(0.2)
        
      # relu activation function
      self.relu =  nn.ReLU()
  
      # dense layer 1
      self.fc1 = nn.Linear(768,512)
        
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)
  
      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, sent_id, mask):
  
        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
        
        x = self.fc1(cls_hs)
        
        x = self.relu(x)
        
        x = self.dropout(x)
        
        # output layer
        x = self.fc2(x)
        
        # apply softmax activation
        x = self.softmax(x)
        
        return x

In [121]:
model = BERT_architecture(bert)
from transformers import AdamW
optimizer = AdamW(model.parameters(),lr = 1e-5) # learning rate


C:\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [122]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [123]:
# function to train the model
def train():
    
    model.train()
    
    total_loss, total_accuracy = 0, 0
    
    # empty list to save model predictions
    total_preds=[]
    
  # iterate over batches
    for step,batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
      # push the batch to gp
        batch = [r.to(device) for r in batch]
        
        sent_id, mask, labels = batch
        
        # clear previously calculated gradients 
        model.zero_grad()        
        
        # get model predictions for the current batch
        preds = model(sent_id, mask)
        
        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)
        
        # add on to the total loss
        total_loss = total_loss + loss.item()
        
        # backward pass to calculate the gradients
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # update parameters
        optimizer.step()
        
        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()
        
        # append the model predictions
        total_preds.append(preds)
    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)
    
    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    total_preds  = np.concatenate(total_preds, axis=0)
    
    #returns the loss and predictions
    return avg_loss, total_preds



In [124]:
# code


# function for evaluating the model
def evaluate():
    print("\nEvaluating...")
    
  # deactivate dropout layers
    model.eval()
    
    total_loss, total_accuracy = 0, 0
    
    # empty list to save the model predictions
    total_preds = []
    # iterate over batches
    for step,batch in enumerate(val_dataloader):
      # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
              # # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
              
      # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
        batch = [t.to(device) for t in batch]
  
        sent_id, mask, labels = batch
        
        # deactivate autograd
        with torch.no_grad():
            # model predictions
            preds = model(sent_id, mask)
            
            # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds,labels)
            
            total_loss = total_loss + loss.item()
            
            preds = preds.detach().cpu().numpy()
            
            total_preds.append(preds)
    avg_loss = total_loss / len(val_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)
    
    return avg_loss, total_preds
            

In [125]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()
	
from sklearn.metrics import classification_report
pred = np.argmax(preds, axis = 1)
print(classification_report(test_y, pred))


              precision    recall  f1-score   support

           0       0.42      0.30      0.35       370
           1       0.56      0.69      0.62       481

    accuracy                           0.52       851
   macro avg       0.49      0.49      0.48       851
weighted avg       0.50      0.52      0.50       851



In [126]:
import gradio as gr

In [127]:

import gradio as gr

def sentiment_analysis(text):
    inputs = tokenizer.encode_plus(text, return_tensors="pt", max_length=17, pad_to_max_length=True, truncation=True)
    sent_id = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    
    model.eval()  # Switch to evaluation mode
    
    with torch.no_grad():
        logits = model(sent_id, mask)
    
    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
    return {"negative": float(probs[0]), "positive": float(probs[1])}


In [128]:
iface = gr.Interface(fn=sentiment_analysis, inputs="text", outputs="label")
iface.launch()

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


C:\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
